# Census release 2025-01-30 (LTS)

In [1]:
!lamin init --storage run-tests --modules bionty

→ initialized lamindb: zethson/run-tests


In [ ]:
import lamindb as ln
import bionty as bt
from cellxgene_lamin.dev import get_datasets_from_cxg, get_collections_from_cxg

ln.settings.verbosity = "hint"

→ connected lamindb: zethson/run-tests


In [3]:
census_version = "2025-01-30"
previous_release = "2024-07-01"

In [4]:
s3path = f"s3://cellxgene-data-public/cell-census/{census_version}/h5ads"
ln.UPath(s3path)

S3QueryPath('s3://cellxgene-data-public/cell-census/2025-01-30/h5ads')

In [5]:
ln.UPath(s3path).view_tree()

0 sub-directories & 1573 files with suffixes '.h5ad'
s3://cellxgene-data-public/cell-census/2025-01-30/h5ads
├── 00099d5e-154f-4a7a-aa8d-fa30c8c0c43c.h5ad
├── 001b01fe-5c70-4bda-a3aa-ee09b7899b14.h5ad
├── 0041b9c3-6a49-4bf7-8514-9bc7190067a7.h5ad
├── 00476f9f-ebc1-4b72-b541-32f912ce36ea.h5ad
├── 00593d17-0693-4646-acad-89dbefba11bb.h5ad
├── 0087cde2-967d-4f7c-8e6e-40e4c9ad1891.h5ad
├── 00ba8341-48ec-4e4e-bb56-be0dd2dd7913.h5ad
├── 00e5dedd-b9b7-43be-8c28-b0e5c6414a62.h5ad
├── 00ff600e-6e2e-4d76-846f-0eec4f0ae417.h5ad
├── 01209dce-3575-4bed-b1df-129f57fbc031.h5ad
├── 0129dbd9-a7d3-4f6b-96b9-1da155a93748.h5ad
├── 015c230d-650c-4527-870d-8a805849a382.h5ad
├── 019c7af2-c827-4454-9970-44d5e39ce068.h5ad
├── 01ad3cd7-3929-4654-84c0-6db05bd5fd59.h5ad
├── 01c93cf6-b695-4e30-a26e-121ae8b16a9e.h5ad
├── 023ce5da-0dbe-4ad0-a290-b6c21824b101.h5ad
├── 023d4d04-4671-4433-af8d-158a05c81d8b.h5ad
├── 02419ebc-fd75-4629-ba13-9b26e3be851c.h5ad
├── 024593dd-d237-4eaa-aff2-9c9d87be595e.h5ad
├── 02792605-4760

In [6]:
ln.track("VzV8Ty89eQdK")

• tracked pip freeze > /home/lukas/.cache/lamindb/environments/run_xkjTYBy2Vsc82WJw/run_env_pip.txt
→ created Transform('VzV8Ty89eQdK0000'), started new Run('xkjTYBy2...') at 2025-07-23 09:25:03 UTC
→ notebook imports: bionty==1.6.1 cellxgene_lamin==0.3.2 lamindb==1.9.0


Get all datasets and associated metadata using cellxgene REST API:

In [7]:
cxg_datasets = get_datasets_from_cxg()
len(cxg_datasets)

1844

In [8]:
cxg_datasets[0].keys()

dict_keys(['assay', 'assets', 'cell_count', 'cell_type', 'citation', 'collection_doi', 'collection_doi_label', 'collection_id', 'collection_name', 'collection_version_id', 'dataset_id', 'dataset_version_id', 'development_stage', 'disease', 'donor_id', 'explorer_url', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'primary_cell_count', 'processing_status', 'published_at', 'revised_at', 'revision_of_collection', 'revision_of_dataset', 'schema_version', 'self_reported_ethnicity', 'sex', 'spatial', 'suspension_type', 'tissue', 'title', 'tombstone', 'visibility', 'x_approximate_distribution'])

## Register artifacts

In [9]:
artifacts = ln.Artifact.from_dir(s3path)
ln.save(artifacts)

→ due to lack of write access, LaminDB won't manage this storage location: s3://cellxgene-data-public
→ referenced read-only storage location at s3://cellxgene-data-public
✓ created 1573 artifacts from directory using storage s3://cellxgene-data-public and key = cell-census/2025-01-30/h5ads/


In [10]:
artifacts = ln.Artifact.filter(key__contains=census_version).all()
len(artifacts)

1573

In [11]:
artifacts.update(version=census_version)

1573

In [12]:
for cxg_dataset in cxg_datasets:
    artifact = artifacts.filter(key__contains=cxg_dataset["dataset_id"]).one_or_none()
    if artifact is not None:
        artifact.n_observations = cxg_dataset["cell_count"]
        artifact.description = cxg_dataset["title"]
        artifact.save()

In [13]:
artifacts_previous = ln.Artifact.filter(version=previous_release).all()
artifacts_previous.count()

0

In [14]:
for artifact in artifacts:
    dataset_id = artifact.key.split("/")[-1]
    artifact_previous = artifacts_previous.filter(
        key__endswith=dataset_id
    ).one_or_none()
    if artifact_previous is not None:
        artifact.add_to_version_family(artifact_previous, version=census_version)

## Register metadata

### Register new features and parent labels

In [15]:
artifacts = ln.Artifact.filter(key__contains=census_version).all()

In [16]:
from cellxgene_lamin.dev._features import register_obs_schema

In [17]:
if not artifacts[0].feature_sets.filter(name="obs metadata").exists():
    obs_feature_set = register_obs_schema(artifacts)

In [18]:
features = ln.Feature.lookup()

## organisms

In [19]:
from cellxgene_lamin.dev._organism import register_organisms, curate_organisms

In [20]:
register_organisms(cxg_datasets)

✓ created 7 Organism records from Bionty matching ontology_id: 'NCBITaxon:9825', 'NCBITaxon:7955', 'NCBITaxon:9598', 'NCBITaxon:9483', 'NCBITaxon:9544', 'NCBITaxon:9606', 'NCBITaxon:10090'


Link collections and organisms to artifacts:

In [21]:
curate_organisms(artifacts, cxg_datasets)

## ontologies

Register all ontology ids:

In [22]:
from cellxgene_lamin.dev._ontology import register_ontology_ids

In [23]:
register_ontology_ids(cxg_datasets)

registering organism

registering assay

✓ created 43 ExperimentalFactor records from Bionty matching ontology_id: 'EFO:0008919', 'EFO:0030007', 'EFO:0700016', 'EFO:0008780', 'EFO:0030074', 'EFO:0030060', 'EFO:0030027', 'EFO:0011025', 'EFO:0010891', 'EFO:0009919', 'EFO:0700003', 'EFO:0030003', 'EFO:0030002', 'EFO:0008877', 'EFO:0030028', 'EFO:0008953', 'EFO:0700004', 'EFO:0009922', 'EFO:0700010', 'EFO:0008931', ...
• did not create ExperimentalFactor records for 2 non-validated ontology_ids: 'EFO:0022845', 'EFO:0022857'


registered 2 records: [ExperimentalFactor(uid='6pMVJ6Yh', name='modified STRT-seq', ontology_id='EFO:0022845', 
branch_id=1, space_id=1, created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object
at 0x75e4f4759100>), ExperimentalFactor(uid='40SRG66e', name='Visium Spatial Gene Expression V1', 
ontology_id='EFO:0022857', branch_id=1, space_id=1, created_by_id=1, run_id=1, 
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e416f2f5c0>)]

registering cell_type

...

✓ created 2 Phenotype records from Bionty matching ontology_id: 'PATO:0000383', 'PATO:0000384'
• did not create Phenotype record for 1 non-validated ontology_id: 'unknown'


registered 1 records: [Phenotype(uid='5HWRj1OD', name='unknown', ontology_id='unknown', branch_id=1, space_id=1, 
created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b1848d70>)]

registering tissue

✓ created 592 Tissue records from Bionty matching ontology_id: 'UBERON:0004167', 'UBERON:0003017', 'UBERON:0002113', 'UBERON:0002116', 'UBERON:0002550', 'UBERON:0005406', 'UBERON:0001809', 'UBERON:0001985', 'UBERON:0002060', 'UBERON:0002657', 'UBERON:0001238', 'UBERON:0001295', 'UBERON:0004262', 'UBERON:0003059', 'UBERON:0013706', 'UBERON:0002351', 'UBERON:0035886', 'UBERON:0004499', 'UBERON:0002328', 'UBERON:0001830', ...
• did not create Tissue records for 14 non-validated ontology_ids: 'CL:0000010', 'CL:0000082', 'CL:0000084', 'CL:0000115', 'CL:0000307', 'CL:0000322', 'CL:0000351', 'CL:0002322', 'CL:0002327', 'CL:0002328', 'CL:0002334', 'CL:0002335', 'CL:0002633', 'CL:4052001'


registered 14 records: [Tissue(uid='2mqTx1qQ', name='epithelial cell of lung', ontology_id='CL:0000082', 
branch_id=1, space_id=1, created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object
at 0x75e4b18c54f0>), Tissue(uid='1LzSyCsJ', name='respiratory basal cell', ontology_id='CL:0002633', branch_id=1, 
space_id=1, created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 
0x75e4b2347e30>), Tissue(uid='XS6m3PvU', name='cultured cell', ontology_id='CL:0000010', branch_id=1, space_id=1, 
created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b233be30>), 
Tissue(uid='1ERsVqRf', name='embryonic stem cell', ontology_id='CL:0002322', branch_id=1, space_id=1, 
created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b25f01d0>), 
Tissue(uid='wxc6eyuU', name='pulmonary alveolar epithelial cell', ontology_id='CL:0000322', branch_id=1, 
space_id=1, created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 
0x75e4b0c38ce0>), Tissue(uid='4IRoehoY', name='preadipocyte', ontology_id='CL:0002334', branch_id=1, space_id=1, 
created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c387a0>), 
Tissue(uid='5F4YNEUS', name='mammary gland epithelial cell', ontology_id='CL:0002327', branch_id=1, space_id=1, 
created_by_id=1, run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c38920>), 
Tissue(uid='1J6s4gSi', name='endothelial cell', ontology_id='CL:0000115', branch_id=1, space_id=1, created_by_id=1,
run_id=1, created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c39490>), 
Tissue(uid='22LvKd01', name='T cell', ontology_id='CL:0000084', branch_id=1, space_id=1, created_by_id=1, run_id=1,
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c39610>), Tissue(uid='MJYdRbdq', 
name='bronchial epithelial cell', ontology_id='CL:0002328', branch_id=1, space_id=1, created_by_id=1, run_id=1, 
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c39880>), Tissue(uid='7MscrSDl', 
name='trophoblast cell', ontology_id='CL:0000351', branch_id=1, space_id=1, created_by_id=1, run_id=1, 
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c39a00>), Tissue(uid='4L12MdXX', 
name='multiciliated ependymal cell', ontology_id='CL:4052001', branch_id=1, space_id=1, created_by_id=1, run_id=1, 
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c39be0>), Tissue(uid='3bgY2Yz8', 
name='tracheal epithelial cell', ontology_id='CL:0000307', branch_id=1, space_id=1, created_by_id=1, run_id=1, 
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c38bf0>), Tissue(uid='8dEt6lpw', 
name='brown preadipocyte', ontology_id='CL:0002335', branch_id=1, space_id=1, created_by_id=1, run_id=1, 
created_at=<django.db.models.expressions.DatabaseDefault object at 0x75e4b0c39dc0>)]

In [24]:
# clean up the celltype in Tissue

bt.Tissue.filter(ontology_id="CL:0000307").delete()
bt.CellType.from_source(ontology_id="CL:0000307").save()

CellType(uid='3bgY2Yz8', name='tracheal epithelial cell', ontology_id='CL:0000307', synonyms='tracheocyte', description='An Epithelial Cell Found In The Trachea.', branch_id=1, space_id=1, created_by_id=1, run_id=1, source_id=16, created_at=2025-07-23 09:26:55 UTC)

## donors and suspension_types

In [25]:
from cellxgene_lamin.dev._labels import register_ulabels

In [26]:
ln.settings.creation.search_names = False
register_ulabels(cxg_datasets, "donor_id")
ln.settings.creation.search_names = True

registered 13075 donor_ids

In [27]:
ln.settings.creation.search_names = False
register_ulabels(cxg_datasets, "suspension_type")
ln.settings.creation.search_names = True

registered 3 suspension_types

## Annotate artifacts with obs metadata

In [28]:
from cellxgene_lamin.dev._features import FEATURE_TO_ACCESSOR

In [29]:
import bionty as bt

features = ln.Feature.lookup()

for idx, cxg_dataset in enumerate(cxg_datasets):
    if idx % 100 == 0:
        print(f"annotating dataset {idx} of {len(cxg_datasets)}")
    artifact = artifacts.filter(key__contains=cxg_dataset["dataset_id"]).one_or_none()
    if artifact is None:
        continue
    for field, terms in cxg_dataset.items():
        if field not in FEATURE_TO_ACCESSOR:
            continue
        accessor, orm = FEATURE_TO_ACCESSOR.get(field)
        if field in ["donor_id", "suspension_type", "tissue_type"]:
            records = orm.from_values(terms, field="name")
            if len(records) > 0:
                # stratify by feature so that link tables records are written
                getattr(artifact, accessor).add(*records)
        else:
            if field == "tissue":
                records = []
                celltypes = [
                    term["ontology_term_id"]
                    for term in terms
                    if term["tissue_type"] == "cell culture"
                ]
                if len(celltypes) > 0:
                    # records += bt.CellType.from_values(
                    #     [i["ontology_term_id"] for i in terms], field="ontology_id"
                    # )
                    print(f"These tissues are not yet linked: {celltypes}")
                tissues = [
                    term["ontology_term_id"]
                    for term in terms
                    if term["tissue_type"] != "cell culture"
                ]
                if len(tissues) > 0:
                    records += bt.Tissue.from_values(
                        [term["ontology_term_id"] for term in terms],
                        field="ontology_id",
                    )
            else:
                records = orm.from_values(
                    [term["ontology_term_id"] for term in terms], field="ontology_id"
                )
            if len(records) > 0:
                getattr(artifact, accessor).add(*records)

# clean up the 2 "unknowns" in DevelopmentalStage
bt.DevelopmentalStage.filter(name="unknown").exclude(ontology_id="unknown").delete()

annotating dataset 0 of 1844
annotating dataset 100 of 1844
✓ loaded 1 Disease record matching ontology_id: 'MONDO:0004981 || MONDO:1030008'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
✓ loaded 3 Disease records matching ontology_id: 'MONDO:0005570', 'MONDO:0018881', 'MONDO:0020077'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
✓ loaded 3 Disease records matching ontology_id: 'MONDO:0005570', 'MONDO:0018881', 'MONDO:0020077'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
• did not create Disease record for 1 non-validated ontology_id: 'PATO:0000461'
✓ loaded 1 Disease record matching ontology_id: 'MONDO:0005002'
• did not create Disease record for 1

## Validate and register genes

In [30]:
from cellxgene_lamin.dev._gene import register_genes

Register all genes for each organism:

In [31]:
register_genes()

✓ created 1 Organism record from Bionty matching name: 'synthetic construct'
✓ created 1 Organism record from Bionty matching ontology_id: 'NCBITaxon:81077'
✓ created 1 Organism record from Bionty matching name: 'sars-2'
✓ created 1 Organism record from Bionty matching ontology_id: 'NCBITaxon:694009'
• key has more than one suffix (path.suffixes), using only last suffix: '.parquet' - if you want your composite suffix to be recognized add it to lamindb.core.storage.VALID_SIMPLE_SUFFIXES.add()
• path content will be copied to default storage upon `save()` with key 'df_synthetic construct__gencode_ercc__1.0.0__Gene.parquet'
✓ storing artifact 'e5fnB0arQ8P1q0Bp0000' at '/home/lukas/code/cellxgene-lamin/docs/notebooks/run-tests/.lamindb/e5fnB0arQ8P1q0Bp0000.parquet'
→ source added!
• key has more than one suffix (path.suffixes), using only last suffix: '.parquet' - if you want your composite suffix to be recognized add it to lamindb.core.storage.VALID_SIMPLE_SUFFIXES.add()
• path content wi

registering homo_sapiens genes

✓ created 62595 Gene records from Bionty matching ensembl_gene_id: 'ENSG00000290825', 'ENSG00000223972', 'ENSG00000227232', 'ENSG00000278267', 'ENSG00000243485', 'ENSG00000284332', 'ENSG00000237613', 'ENSG00000268020', 'ENSG00000290826', 'ENSG00000240361', 'ENSG00000186092', 'ENSG00000238009', 'ENSG00000239945', 'ENSG00000233750', 'ENSG00000268903', 'ENSG00000269981', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000222623', 'ENSG00000241599', ...
• did not create Gene records for 159 non-validated ensembl_gene_ids: 'ENSG00000203441', 'ENSG00000214783', 'ENSG00000214970', 'ENSG00000215067', 'ENSG00000223458', 'ENSG00000223797', 'ENSG00000224167', 'ENSG00000224247', 'ENSG00000225205', 'ENSG00000226032', 'ENSG00000226277', 'ENSG00000226362', 'ENSG00000226747', 'ENSG00000226822', 'ENSG00000226849', 'ENSG00000227925', 'ENSG00000228135', 'ENSG00000228434', 'ENSG00000228890', 'ENSG00000229611', ...
✓ 62595 unique terms (99.70%) are validated for ensembl_gene_id
! 159 unique terms (0.30%) are 

registering mus_musculus genes

✓ created 56867 Gene records from Bionty matching ensembl_gene_id: 'ENSMUSG00000102693', 'ENSMUSG00000064842', 'ENSMUSG00000051951', 'ENSMUSG00000102851', 'ENSMUSG00000103377', 'ENSMUSG00000104017', 'ENSMUSG00000103025', 'ENSMUSG00000089699', 'ENSMUSG00000103201', 'ENSMUSG00000103147', 'ENSMUSG00000103161', 'ENSMUSG00000102331', 'ENSMUSG00000102348', 'ENSMUSG00000102592', 'ENSMUSG00000088333', 'ENSMUSG00000102343', 'ENSMUSG00000025900', 'ENSMUSG00000102948', 'ENSMUSG00000104123', 'ENSMUSG00000025902', ...
• did not create Gene records for 74 non-validated ensembl_gene_ids: 'ENSMUSG00000043623', 'ENSMUSG00000046145', 'ENSMUSG00000046388', 'ENSMUSG00000046747', 'ENSMUSG00000047189', 'ENSMUSG00000048316', 'ENSMUSG00000048406', 'ENSMUSG00000049202', 'ENSMUSG00000052005', 'ENSMUSG00000052241', 'ENSMUSG00000052426', 'ENSMUSG00000052779', 'ENSMUSG00000054304', 'ENSMUSG00000054379', 'ENSMUSG00000054957', 'ENSMUSG00000055216', 'ENSMUSG00000055872', 'ENSMUSG00000055958', 'ENSMUSG00000055968', 'E

registering synthetic_construct genes

✓ created 92 Gene records from Bionty matching ensembl_gene_id: 'ERCC-00002', 'ERCC-00003', 'ERCC-00004', 'ERCC-00009', 'ERCC-00012', 'ERCC-00013', 'ERCC-00014', 'ERCC-00016', 'ERCC-00017', 'ERCC-00019', 'ERCC-00022', 'ERCC-00024', 'ERCC-00025', 'ERCC-00028', 'ERCC-00031', 'ERCC-00033', 'ERCC-00034', 'ERCC-00035', 'ERCC-00039', 'ERCC-00040', ...
✓ 92 unique terms (100.00%) are validated for ensembl_gene_id


registering severe_acute_respiratory_syndrome_coronavirus_2 genes

✓ created 12 Gene records from Bionty matching ensembl_gene_id: 'ENSSASG00005000002', 'ENSSASG00005000003', 'ENSSASG00005000004', 'ENSSASG00005000006', 'ENSSASG00005000010', 'ENSSASG00005000007', 'ENSSASG00005000011', 'ENSSASG00005000009', 'ENSSASG00005000012', 'ENSSASG00005000008', 'ENSSASG00005000005', 'ENSSASG00005000013'
✓ 12 unique terms (100.00%) are validated for ensembl_gene_id


## Link metadata to individual artifacts

annotate with genes measured in each artifact:

In [32]:
organisms = bt.Organism.lookup(field=bt.Organism.scientific_name)

In [33]:
# TODO discuss whether we even want to keep doing this
for idx, artifact in enumerate(artifacts):
    if idx % 100 == 0:
        print(f"annotating dataset {idx} of {len(artifacts)}")

    adata = artifact.open()
    var_names = adata.var_names
    organism_record = artifact.organisms.first()
    if organism_record is None:
        print(f"No organism found for artifact: {artifact}")
        continue

    genes = bt.Gene.from_values(
        var_names, field=bt.Gene.ensembl_gene_id, organism=organism_record
    )

    if len(genes) == 0 and var_names[0].startswith("ENSG"):
        genes = bt.Gene.from_values(
            var_names, field=bt.Gene.ensembl_gene_id, organism="human"
        )

    if len(var_names[var_names.str.startswith("ERCC")]) > 0:
        genes = bt.Gene.from_values(
            var_names,
            field=bt.Gene.ensembl_gene_id,
            organism=organisms.synthetic_construct,
        )
    if len(var_names[var_names.str.startswith("ENSSASG")]) > 0:
        genes = bt.Gene.from_values(
            var_names,
            field=bt.Gene.ensembl_gene_id,
            organism=organisms.severe_acute_respiratory_syndrome_coronavirus_2,
        )

    if genes:
        genes = [g for g in genes if not g._state.adding]
        var_schema = ln.Schema(genes, dtype="number").save()
        artifact.feature_sets.add(var_schema, through_defaults={"slot": "var"})

annotating dataset 0 of 1573
✓ loaded 36402 Gene records matching ensembl_gene_id: 'ENSG00000243485', 'ENSG00000237613', 'ENSG00000186092', 'ENSG00000238009', 'ENSG00000239945', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000241599', 'ENSG00000286448', 'ENSG00000236601', 'ENSG00000284733', 'ENSG00000235146', 'ENSG00000284662', 'ENSG00000229905', 'ENSG00000237491', 'ENSG00000177757', 'ENSG00000228794', 'ENSG00000225880', 'ENSG00000230368', 'ENSG00000272438', ...
✓ created 4 Gene records from Bionty matching ensembl_gene_id: 'ENSG00000224516', 'ENSG00000225489', 'ENSG00000285517', 'ENSG00000286061'
✓ loaded 61752 Gene records matching ensembl_gene_id: 'ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938', 'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084', 'ENSG00000001167', 'ENSG00000001460', 'ENSG00000001461', 'ENSG00000001497', 'ENSG00000001561', 'ENSG00000001617', 'ENSG00000001626', 'ENSG00000001629', 'ENSG00000001630',

KeyboardInterrupt: 

In [34]:
artifact.describe()

Artifact .h5ad · AnnData
├── General
│   ├── key: cell-census/2025-01-30/h5ads/2104fbb8-8ce3-4740-8b6a-bcbb46a13c0f.h5ad
│   ├── description: Visium spatial - HCAHeartST10238252 (OCT)
│   ├── uid: qsj12UDM6qYq4bN60000          hash: xX9Blm7gJgiq8KfRbG2LXw
│   ├── size: 323.0 MB                     transform: cencus-release-2025-01-30-LTS.ipynb
│   ├── space: all                         branch: all
│   ├── created_by: zethson                created_at: 2025-07-23 11:26:00
│   ├── n_observations: 4992               version: 2025-01-30
│   └── storage path: 
│       s3://cellxgene-data-public/cell-census/2025-01-30/h5ads/2104fbb8-8ce3-4740-8b6a-bcbb46a13c0f.h5ad
├── Dataset features
│   └── obs • 11                        [Feature]                                                                  
│       organism                        cat[bionty.Organism]                                                       
│       assay                           cat[bionty.ExperimentalFactor]                                             
│       cell_type                       cat[bionty.CellType]                                                       
│       development_stage               cat[bionty.DevelopmentalStage]                                             
│       disease                         cat[bionty.Disease]                                                        
│       self_reported_ethnicity         cat[bionty.Ethnicity]                                                      
│       sex                             cat[bionty.Phenotype]                                                      
│       tissue                          cat[bionty.Tissue]                                                         
│       suspension_type                 cat[ULabel]                                                                
│       donor_id                        cat[ULabel]                                                                
│       tissue_type                     cat[ULabel]                                                                
└── Labels
    └── .organisms                      bionty.Organism                    human                                   
        .tissues                        bionty.Tissue                      left cardiac atrium                     
        .cell_types                     bionty.CellType                    vein endothelial cell, CD16-positive, C…
        .phenotypes                     bionty.Phenotype                   male                                    
        .experimental_factors           bionty.ExperimentalFactor          Visium Spatial Gene Expression V1       
        .developmental_stages           bionty.DevelopmentalStage          fifth decade stage                      
        .ethnicities                    bionty.Ethnicity                   European                                
        .ulabels                        ULabel                             D8, na

## Annotate tissue_type

In [35]:
register_ulabels(cxg_datasets, "tissue_type")

! you are trying to create a record with name='tissue' but a record with similar name exists: 'is_tissue_type'. Did you mean to load it?
! you are trying to create a record with name='organoid' but records with similar names exist: 'Chan_NatCommun_2022_bronchial_organoids', 'Lim_CellStemCell_2023_Lung_organoid_epithelium', 'Wesley_NatCellBio_2022_Hepatoblast_organoids_treated_with_hepatozyme'. Did you mean to load one of them?


## Register collections

In [ ]:
collection = ln.Collection(
    artifacts,
    key="cellxgene-census",
    version=census_version,
    is_new_version_of=ln.Collection.filter(
        key="cellxgene-census", version=previous_release
    ).one(),
)
collection.save()

Collection(uid='1eshvBBOwC0IdzGZ0000', version='2025-01-30', is_latest=True, key='cellxgene-census', hash='NjqvY0g6hlzgyVXTYer0Ng', branch_id=1, space_id=1, created_by_id=1, run_id=1, created_at=2025-07-23 13:29:50 UTC)

In [37]:
cxg_collections = get_collections_from_cxg()

In [39]:
artifacts.first().key

'cell-census/2025-01-30/h5ads/00099d5e-154f-4a7a-aa8d-fa30c8c0c43c.h5ad'

In [40]:
ln.settings.creation.search_names = False
for collection_meta in cxg_collections:
    keys = [
        f"cell-census/{census_version}/h5ads/{dataset['dataset_id']}.h5ad"
        for dataset in collection_meta["datasets"]
    ]
    collection_artifacts = artifacts.filter(key__in=keys).all()
    if collection_artifacts.count() > 0:
        kwargs = {
            "key": collection_meta["name"],
            "description": collection_meta["doi"],
            "reference": collection_meta["collection_id"],
            "reference_type": "CELLxGENE Collection ID",
            "version": census_version,
        }
        collection_record = ln.Collection(
            collection_artifacts,
            **kwargs,
        )
        # if is needed here as .save() errors if collection is already saved
        if collection_record._state.adding:
            collection_record.save()
ln.settings.creation.search_names = True

Add existing collections to their corresponding version families:

In [41]:
collections = ln.Collection.filter(version=census_version).all()
collections.count()

248

In [42]:
collections_previous = ln.Collection.filter(version=previous_release).all()
collections_previous.count()

0

In [43]:
for collection in collections:
    collection_previous = collections_previous.filter(
        reference=collection.reference
    ).one_or_none()
    if collection_previous is not None:
        collection.add_to_version_family(collection_previous, version=census_version)

## Link metadata to collection

In [44]:
collection = ln.Collection.filter(key="cellxgene-census", version=census_version).one()
collection

Collection(uid='1eshvBBOwC0IdzGZ0000', version='2025-01-30', is_latest=True, key='cellxgene-census', hash='NjqvY0g6hlzgyVXTYer0Ng', branch_id=1, space_id=1, created_by_id=1, run_id=1, created_at=2025-07-23 13:29:50 UTC)

feature sets:

In [45]:
collection.describe()

''

## Register the soma store

In [46]:
soma_path = f"s3://cellxgene-data-public/cell-census/{census_version}/soma"
ln.UPath(soma_path).view_tree()

19 sub-directories & 7 files with suffixes '', '.tdb'
s3://cellxgene-data-public/cell-census/2025-01-30/soma
├── __tiledb_group.tdb
├── __group/
│   └── __1738275389572_1738275389572_13353233232d51777852409560eaff8d_2
├── __meta/
│   ├── __1738275389508_1738275389508_4e0e75839417b5b157905a31316816e6
│   └── __1738275389572_1738275389572_2639dfe80b470afbf96934cfd352226e
├── census_data/
│   ├── __tiledb_group.tdb
│   ├── __group/
│   ├── __meta/
│   ├── homo_sapiens/
│   └── mus_musculus/
├── census_info/
│   ├── __tiledb_group.tdb
│   ├── __group/
│   ├── __meta/
│   ├── datasets/
│   ├── organisms/
│   ├── summary/
│   └── summary_cell_counts/
└── census_spatial_sequencing/
    ├── __tiledb_group.tdb
    ├── __group/
    ├── __meta/
    ├── homo_sapiens/
    └── mus_musculus/


In [47]:
soma_artifact = ln.Artifact(soma_path, description=f"Census {census_version}").save()
soma_artifact

• path in storage 's3://cellxgene-data-public' with key 'cell-census/2025-01-30/soma'


Artifact(uid='DstoXzgJPW2PrnYr0000', is_latest=True, key='cell-census/2025-01-30/soma', description='Census 2025-01-30', suffix='', size=1128639499732, hash='uq_4QzGnaveTq5XinBJiCA', n_files=14217, branch_id=1, space_id=1, storage_id=2, run_id=1, created_by_id=1, created_at=2025-07-23 13:32:09 UTC)

In [48]:
ln.finish()

• please hit CTRL + s to save the notebook in your editor . ✓
! cells [(35, None), (None, 37), (37, 39)] were not run consecutively
→ finished Run('xkjTYBy2') after 4h at 2025-07-23 13:32:12 UTC
